# Метод простой итерации, задание А

$Ax=b$

$tAx = tb$

$x + tAx = x + tb$

$x = (E - tA)x + tb$

$x = Cx + d$

1) Если $A = A^*$, $A > 0$, то $t_{опт} = \frac 2 {\lambda_{min} + \lambda_{max}}$, и МПИ сходится.

2) Если $A$ имеет диагональное преобладание, то $C= E − D^{-1}A$, $d = D^{-1}b$,
где D — диагональная матрица, у которой на диагонали стоят диагональные элементы матрицы A.

In [1]:
import numpy as np
import numpy.linalg as linalg

import warnings
warnings.filterwarnings('ignore')

In [25]:
def transform(A, b, D):
    n = A.shape[0]
    C = np.eye(n) - D @ A
    d = D @ b.reshape(-1,1)
    return C, d

In [26]:
D = np.array([[1., -0.2, 0.1],
              [-0.1, 1.0, -0.1],
             [-0.3, 0.2, -1.]])
G = 0.1 * np.eye(3)

In [27]:
k = 4
A = D + k * G
b = np.array([0.4, 0.8, 0.2])

In [28]:
A

array([[ 1.4, -0.2,  0.1],
       [-0.1,  1.4, -0.1],
       [-0.3,  0.2, -0.6]])

Так как у нас диагональ преобладает.

In [6]:
D = A * np.eye(3)

In [7]:
C, d = transform(A, b, linalg.inv(D))

In [8]:
C_norm = linalg.norm(C, ord=np.inf)
C_norm

0.8333333333333334

In [9]:
linalg.eigvalsh(C)

array([-0.63536368,  0.06579317,  0.5695705 ])

In [10]:
def do_iter(C, d, k):
    x = np.ones(shape=(3,1))
    for i in range(k):
        x = C @ x + d
    return x

In [11]:
x_15 = do_iter(C, d, 15)
x_15

array([[ 0.39201451],
       [ 0.57531761],
       [-0.33756807]])

In [12]:
def calc_upper_est(C, d, k):
    x_0 = do_iter(C, d, 0)
    x_1 = do_iter(C, d, 1)
    C_norm = linalg.norm(C, ord=np.inf)
    return (C_norm ** k) * linalg.norm(x_1 - x_0, ord=np.inf) / (1 - C_norm)

In [13]:
calc_upper_est(C, d, 15)

0.5841492436698708

In [14]:
def find_k(C, d, e):
    k = 0
    est = calc_upper_est(C, d, k)
    while est >= e:
        k += 1
        est = calc_upper_est(C, d, k)
    return k

In [15]:
find_k(C, d, 10**-5)

76

In [16]:
calc_upper_est(C, d, 76)

8.639086248605786e-06

In [17]:
def get_lambda(C, d, k):
    x_k = do_iter(C, d, k)
    x_k_1 = do_iter(C, d, k-1)
    x_k_2 = do_iter(C, d, k-2)
    y_k = x_k - x_k_1
    y_k_1 = x_k_1 - x_k_2
    i = np.argmax(abs(y_k))
    return y_k[i] / y_k_1[i]

In [18]:
def lusternik(C, d, k):
    x_k = do_iter(C, d, k)
    x_k_1 = do_iter(C, d, k-1)
    l = get_lambda(C, d, 20)
    return x_k + l /  (1 - l) * (x_k - x_k_1)

In [19]:
lusternik(C, d, 76)

array([[ 0.39201452],
       [ 0.5753176 ],
       [-0.33756806]])

# Метод Зейделя, задание А

In [20]:
A = np.array([[2.20886, 0.31984, 0.15751],
             [0.31984, 3.18182, 0.52629],
             [0.15751, 0.52629, 4.98873]])
b = np.array([2.18310, 6.63605, 6.44335])

In [29]:
def zeydel_transform(A, b):
    D = np.eye(3) * A
    R = np.array([[1, 1, 1],
                 [0, 1, 1],
                 [0, 0, 1]]) * A
    C = -linalg.inv(D + R.T) @ R
    d = -linalg.inv(D + R.T) @ b.reshape(-1, 1)
    return C, d

In [30]:
C, d = zeydel_transform(A, b)

In [31]:
linalg.norm(C, ord=np.inf)

0.6071428571428571

In [24]:
calc_upper_est(C, d, k=20)

0.0003130117733854205

# Метод простой итерации с оптимальным параметром, задание А

In [33]:
def gershgorin_borders(A):
    n = A.shape[0]
    ms = []
    Ms = []
    for i in range(n):
        s = 0
        for j in range(n):
            if i == j:
                continue
            s += abs(A[i][j])
        Ms.append(s + A[i][i])
        ms.append(-s + A[i][i])
    return min(ms), max(Ms)

In [37]:
m, M = gershgorin_borders(A)
m, M

(1.73151, 5.67253)

In [38]:
t_opt = 2 / (m + M)

In [39]:
def transform_opt(A, b, t):
    n = A.shape[0]
    C = np.eye(n) - t * A
    d = t * b.reshape(-1,1)
    return C, d

In [40]:
C, d = transform_opt(A, b, t_opt)

In [42]:
linalg.norm(C, ord=np.inf)

0.5322796743399552

In [49]:
def find_est_e(C, d, e):
    k = find_k(C, d, e)
    return do_iter(C, d, k), k

In [50]:
find_est_e(C, d, 10**-3)

(array([[0.64475319],
        [1.84268472],
        [1.07682539]]), 12)

In [47]:
def find_teor_k(M, m, e):
    return np.log(e) / np.log((M - m) / (M + m))

In [48]:
find_teor_k(M, m, 10**-3)

10.954497600288681